<a href="https://colab.research.google.com/github/sophieaea/node_test1/blob/main/junho.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import numpy as np
import matplotlib.pyplot as plt
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

plt.figure(figsize=(1, 1))

def create_graded_line(angle_degrees, img_size=100):
    # 빈 이미지 생성
    img = np.ones((img_size, img_size)) * 255
    img = np.uint8(img)

    draw_line(angle_degrees)

    return img

def get_intersection_points(angle_degrees, img_size=100):
    angle_radians = np.deg2rad(angle_degrees)
    tan_value = np.tan(angle_radians)

    intersections = []

    x_top = img_size / 2 - (img_size / 2) / tan_value
    if 0 <= x_top <= img_size:
        intersections.append((x_top, img_size))

    x_bottom = img_size / 2 + (img_size / 2) / tan_value
    if 0 <= x_bottom <= img_size:
        intersections.append((x_bottom, 0))

    y_left = img_size / 2 - img_size / 2 * tan_value
    if 0 <= y_left <= img_size:
        intersections.append((0, y_left))

    y_right = img_size / 2 + img_size / 2 * tan_value
    if 0 <= y_right <= img_size:
        intersections.append((img_size, y_right))

    # 교차점이 2개 초과인 경우 필요한 교차점만 선택
    if len(intersections) > 2:
        # 각도가 45도 또는 -45도인 경우 대각선 교차점 선택
        if angle_degrees == 45:
            return [intersections[1], intersections[2]]
        elif angle_degrees == -45:
            return [intersections[0], intersections[3]]
        elif 0 < angle_degrees < 90:
            return [intersections[2], intersections[3]]
        elif -90 < angle_degrees < 0:
            return [intersections[0], intersections[1]]
    return intersections

def draw_line(angle_degrees, img_size=100):
    intersections = get_intersection_points(angle_degrees, img_size)
    if len(intersections) != 2:
        print("올바르지 않은 교차점입니다.")
        return

    start, end = intersections

    # 여기에서 color='black'을 추가하여 직선의 색상을 검정으로 설정
    plt.plot([start[0], end[0]], [start[1], end[1]], label=f'{angle_degrees}°', color='black')
    #plt.legend()
    plt.tight_layout()  # 여백 제거
    plt.axis('off')  # 축 제거

    plt.xlim(0, img_size)
    plt.ylim(0, img_size)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.gca().invert_yaxis()  # 이미지 좌표계와 일치시키기 위해 y축을 뒤집습니다.
     # 서브플롯의 여백을 0으로 조정
    plt.subplots_adjust(top=1, bottom=0, right=1, left=0, hspace=0, wspace=0)
    plt.margins(0, 0)
    plt.gca().xaxis.set_major_locator(plt.NullLocator())
    plt.gca().yaxis.set_major_locator(plt.NullLocator())
    #plt.grid(True)

def save_image_to_drive(img, filename, folder_id):
    # 일시적으로 이미지 저장
    plt.savefig(filename, dpi=100, bbox_inches='tight')

    # 저장한 이미지를 구글 드라이브에 업로드
    file = drive.CreateFile({
        'title': filename,
        'parents': [{'id': folder_id}]
    })
    file.SetContentFile(filename)
    file.Upload()
    # 로컬에서 일시 저장한 이미지 삭제
    os.remove(filename)


def draw_and_save_line(img, angle_degrees, folder_id=None):

    if folder_id:
        filename = f"line_{angle_degrees}.png"
        save_image_to_drive(img, filename, folder_id)
        plt.clf()

# Google Drive 인증
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

folder_id = '116aMuleNnn0OCLnBcPEyqPxgcTwwahAh'  # Google Drive 폴더 ID를 여기에 입력하세요.

for angle in range(0, 181, 1):  # 0도부터 179도까지 1도 간격으로 직선 이미지 생성 및 저장
    img = create_graded_line(angle)
    filename = f"line_{angle}.png"
    draw_and_save_line(img, angle, folder_id)


<ipython-input-12-e22da15148f1>:26: RuntimeWarning: divide by zero encountered in double_scalars
  x_top = img_size / 2 - (img_size / 2) / tan_value
<ipython-input-12-e22da15148f1>:30: RuntimeWarning: divide by zero encountered in double_scalars
  x_bottom = img_size / 2 + (img_size / 2) / tan_value


<Figure size 100x100 with 0 Axes>